In [2]:
import xclim.indicators.atmos as xci
import xarray as xr
import glob
import os
import numpy as np

In [3]:
nc_file_list = glob.glob(os.path.join('./raw/pr', "*.nc"))
combined_ds = xr.open_mfdataset(nc_file_list, combine='by_coords')

In [4]:
monthly_ds_spi = xci.standardized_precipitation_index(
    combined_ds.pr, 
    window=3, 
    freq='MS',
    dist="gamma",
    method="APP",
    cal_end="2010-12-30",
    fitkwargs={"floc": 0},
)

monthly_ds_spi

c:\Users\RYAN.DEMELLO\Code\climate-data-viewer\.venv\Lib\site-packages\xclim\core\cfchecks.py:79: UserWarning: Variable has a non-conforming standard_name: Got `lwe_precipitation_rate`, expected `['precipitation_flux']`
  check_valid(vardata, "standard_name", data["standard_name"])
c:\Users\RYAN.DEMELLO\Code\climate-data-viewer\.venv\Lib\site-packages\xclim\indices\stats.py:977: UserWarning: The input data is chunked on time dimension and must be fully rechunked to run `fit` on groups . Beware, this operation can significantly increase the number of tasks dask has to handle.
  da, _ = preprocess_standardized_index(da, freq=freq, window=window, **indexer)


<xarray.DataArray 'spi' (ensemble_member: 1, time: 1200,
                         projection_y_coordinate: 244,
                         projection_x_coordinate: 180)> Size: 422MB
dask.array<where, shape=(1, 1200, 244, 180), dtype=float64, chunksize=(1, 1, 122, 90), chunktype=numpy.ndarray>
Coordinates:
  * ensemble_member          (ensemble_member) int32 4B 4
  * projection_y_coordinate  (projection_y_coordinate) float64 2kB -3.25e+04 ...
  * projection_x_coordinate  (projection_x_coordinate) float64 1kB -1.975e+05...
    ensemble_member_id       (ensemble_member) |S27 27B dask.array<chunksize=(1,), meta=np.ndarray>
    latitude                 (projection_y_coordinate, projection_x_coordinate) float64 351kB dask.array<chunksize=(244, 180), meta=np.ndarray>
    longitude                (projection_y_coordinate, projection_x_coordinate) float64 351kB dask.array<chunksize=(244, 180), meta=np.ndarray>
    prob_of_zero             (ensemble_member, time, projection_y_coordinate, projection_x_coordinate) float64 422MB dask.array<chunksize=(1, 1200, 122, 90), meta=np.ndarray>
  * time                     (time) object 10kB 1980-12-01 00:00:00 ... 2080-...
Attributes: (12/13)
    calibration_period:  ('1980-12-01', '2010-12-01')
    freq:                MS
    window:              3
    scipy_dist:          gamma
    method:              APP
    group:               time.month
    ...                  ...
    time_indexer:        {}
    cell_methods:        time: mean
    history:             [2025-08-04 14:02:40] spi: xclim.core.indicator.SPI(...
    standard_name:       spi
    long_name:           Standardized precipitation index (spi)
    description:         Precipitations over a moving 3-x window, normalized ...

In [5]:
months_below_minus1 = monthly_ds_spi < -2
yearly_avg_spi = months_below_minus1.groupby('time.year').mean(dim='time', keep_attrs=True)
yearly_avg_spi = yearly_avg_spi * 12
# cumulative_below = months_below_minus1.cumsum(dim='time')

# # Create an array of month indices (starting from 1 to avoid division by zero)
# month_indices = xr.DataArray(np.arange(1, cumulative_below.sizes['time'] + 1), dims='time', coords={'time': cumulative_below['time']})

# # Divide cumulative_below by the month index
# cumulative_below_normalized = cumulative_below / month_indices

# cumulative_below_normalized= cumulative_below_normalized.groupby('time.year').mean(dim='time', keep_attrs=True)

# cumulative_below_normalized.to_netcdf('cumalative_test_2_cal.nc')


In [ ]:
# Calculate the cumulative sum of months below threshold along the time dimension
cumulative_below = months_below_minus1.cumsum(dim='time')

# For each year, find the index of the last time in that year
# Get the time coordinate as a DataArray
time_coord = months_below_minus1['time']
years = time_coord.dt.year

# Find the indices of the last time for each year
_, last_indices = np.unique(years.values, return_index=True)
last_indices = np.roll(last_indices, -1) - 1
last_indices[-1] = len(time_coord) - 1  # fix last index

# Select the cumulative sum at the end of each year
cumulative_below_yearly = cumulative_below.isel(time=xr.DataArray(last_indices, dims='year'))

# The number of months passed up to each year is just the index + 1
months_passed = xr.DataArray(last_indices + 1, dims='year', coords={'year': years.values[last_indices]})

# Calculate the ratio
likelihood_ratio = cumulative_below_yearly / months_passed

likelihood_ratio

In [ ]:
rolling_spi = yearly_avg_spi.rolling(year=30, center=True, min_periods=1).mean()
# rolling_spi.to_netcdf('rolling_spi_cal.nc')